# Introdução ao PyTorch


[PyTorch](http://pytorch.org/) é um framework para desenolver e treinar redes neurais. Muitas de suas funções se comportam exatamente da mesma forma que o numpy, onde os arrays são chamados de tensores. A vantagem desses tensores em relação aos arrays do numpy é que eles facilitam a movimentação dos dados da CPU para a GPU, e também são usados por funções do PyTorch para computar altomaticamente gradientes (para o backpropagation) e outros modulos para construir redes neurais. No geral, PyTorch é mais coerente com programação Python e Numpy/Scipy quando comparada com TensorFlow ou outros frameworks.


Como vimos na Regressão Linear, Regressão Logística e Perceptron, uma coisa muito comum na área de aprendizado de máquinas é resolver equações lineares do tipo:

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

Em forma de vetores, podemos representar o produto escalar:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

## Tensores

A grosso modo, algoritmos de aprendizado envolvem um monte de operações de algebra linear em tensores, uma generalização de matrizes. Um vetor é um tensor de 1 dimensão, e uma matriz é um tensor de 2 dimensões, e um array com 3 dimensões é um tensor tridimensional (para imagens com RGB, por exemplo). Sendo assim, a estrutura fundamental de redes neurais e PyTorch são os tensores.


<img src="assets/tensor_examples.svg" width=600px>

Vejamos alguns exemplos de uso do PyTorch e seus tensores

In [1]:
# Primeiramente, importamos o PyTorch
import torch

In [2]:
def activation(x):
    """ Função de ativação - Sigmoid
    
        Argumentos
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [3]:
### Gerando dados aleatórios
torch.manual_seed(7) # Seta um seed para sempre gerar os mesmos números aleatórios

# Features é um tensor com 1 linha (1 única amostra) e 5 colunas (5 características por amostra), 
#   inicializadas de forma aleatória usando uma distribuição normal com média zero e desvio 1.
features = torch.randn((1, 5))

# Gerando os pesos aleatórios para o nosso modelo. randn_like gera um tensor com as mesmas carácterísticas
#    que o tensor passado como parâmetro
weights = torch.randn_like(features)


# termo de bias - tensor com uma única linha e coluna.
bias = torch.randn((1, 1))

Esses tensores podem ser somados, subtraidos, multiplicados, etc, assim como os arrays numpy. Em geral, usamos os tensores de forma bem parecida com esses arrays, com a vantagem de poder utilizar em GPUs. Como exemplo, podemos computar a saída desse nosso modelo de neurônio:

In [1]:
# Podemos predizer a saída do nosso neurônio:

# Assim como numpy, podemos usar as opções torch.sum(), assim como o métodos .sum() nos tensores.

# opção 1 - torch.sum()
y = activation(torch.sum(features * weights) + bias)
print('opção 1: ', y)

# opção 2 - .sum()
y = activation((features * weights).sum() + bias)
print('opção 2: ', y)

# Podemos também juntar as operações de soma e multiplicação numa única operação, executando a multiplicação
#   de matrizes. Em geral, multiplicação de matriz é mais eficiente, principalmente em GPUs. 
#   Para tanto, podemos utilizar as funções torch.mm() ou torch.matmul() - a última é mais complicada
#      e oferece mais opções (verificar em https://pytorch.org/docs/stable/generated/torch.matmul.html).

# opção 3 - torch.mm()
y = activation(torch.mm(features, weights.view(5,1)) + bias)
print('opção 3: ', y)

Note que na opção 3 tivemos que redimensionar nosso vetor de pesos com a função .view(). Caso contrário, teríamos um erro muito comum:

In [2]:
torch.mm(features, weights)

Isso por que, na multiplicação de matrizes, o número de colunas no primeiro tensor deve ser igual ao número de linhas do segundo tensor. Tanto o tensor features quanto o tensor weights tem o mesmo formato, i.e., (1,5). Sendo assim, foi necessário mudar o formato para que a multiplicação funcionasse.

Obs: para ver o formato de um tensor, podemos simplesmente usar .shape.

Existem também algumas opções para mudar o formato do tensor: [`.reshape()`](https://pytorch.org/docs/stable/generated/torch.reshape.html#torch.reshape), [`.resize_()`](https://pytorch.org/docs/stable/generated/torch.Tensor.resize_.html#torch.Tensor.resize_) e [`.view()`](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html#torch.Tensor.view).

Usando como exemplo nosso tensor weights:

* `weights.reshape(a, b)` retorna um novo tensor com os mesmos dados de `weights` com tamanho `(a, b)`, quando possível retorna apenas um view(), quando não, faz uma cópia dos dados.

* `weights.resize_(a, b)` retorna o mesmo tensor com um formato diferente. Se o número de elementos for menor que o original, alguns elementos serão removidos do tensor (mas não da memória). Se o novo formato tem mais elementos, estes serão inicializados na memória. Note que o _ quer dizer que as operações são executadas **in-place**. [Clique aqui para mais informações a respeito](https://discuss.pytorch.org/t/what-is-in-place-operation/16244).

* `weights.view(a, b)` retorna um tensor  no formato `(a, b)` com os mesmos dados contidos wm `weights`.



## Numpy para Torch e vice-versa

PyTorch apresenta diversos modos de converter entre arrays Numpy e tensores Torch. Vejamos alguns exemplos:

In [3]:
# Gerando um array numpy de (4,3)
import numpy as np
a = np.random.rand(4,3)
a

In [4]:
# Convertendo de numpy para tensor
b = torch.from_numpy(a)
b

In [5]:
# convertendo de tensor para numpy
b.numpy()

A memória é compartilhada entre o array Numpy e o tensor Torch, então, se o valor de um objeto for mudado _in-place_ , o outro objeto também será alterado.

In [6]:
# Multiplicando o tensor por 2, in-place
b.mul_(2)

In [7]:
# O array numpy é ajustado ao novo valor
a

## Regressão linear com PyTorch

<img src="assets/rl.png" width=600px>

In [11]:
# Passo n°1: importando pacotes
import torch
from torch.autograd import Variable
from torch.nn import functional as F

In [8]:
# Verificando a versão do PyTorch e se está usando GPU
print('Versão PyTorch: ', torch.__version__)
print('Usando GPU: ', torch.cuda.is_available())

if torch.cuda.is_available():    
    print('GPU: ',torch.cuda.get_device_name(torch.cuda.current_device()))

In [13]:
# Passo n°2: gerando algumas amostras
x_data = Variable(torch.Tensor([[10.0], [9.0], [3.0], [2.0]]))
y_data = Variable(torch.Tensor([[90.0], [80.0], [50.0], [30.0]]))

Para configurar a classe do nosso modelo, precisamos definir a classe init (onde definimos os atributos) e forward (em redes neurais é muito comum, visto que a etapa de atualização dos pesos é feita de modo backward com o backpropagation). 

Como o modelo recebe como entrada uma amostra com uma única _feature_ e entrega como saída um único valor, inicializamos o modelo com uma camada linear: torch.nn.Linear(1, 1). Linear por que essa camada executa uma combinação linear (os pesos e bias são intrínsicos na camada e não precisamos definir). O primeiro 1 representa o número de características de entrada, e o segundo 1 significa o tamanho da saída.

Em seguida, definimos a função forward, que basicamente contém as instruções da sequência dos passos do modelo. Em outras palavras, esse passo executa todos os processos do modelo desde os dados de entrada até a saída. Como a regressão linear é bem simples, a função recebe uma entrada $x$ e produz uma estimativa de $y$ como saída, ou seja, $\hat{y}$.

In [14]:
class LinearRegression(torch.nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(1, 1)
        
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred
    
model = LinearRegression()

## Função de Loss (Criterion) e Otimizador

Após executar a função forward, a função de loss é usada para computar o quão distante está $\hat{y}$ de $y$, e assim ajustar os pesos para aproximar essa diferença, a fim de produzir o melhor modelo possível. Definir essa função de loss no PyTorch é muito simples. Nesse caso usaremos o erro médio quadrado ( _Mean Square Error (MSE)_ ), por ser mais comum na tarefa de regressão.

In [15]:
criterion = torch.nn.MSELoss()

Na sequência, usaremos o otimizador Gradiente Descendente Estocástico (_Stochastic Gradient Descent (SGD)_) para atualizar os pesos do modelo. A função model.parameters() diz ao otimizador quais são os pesos a serem atualizados, enquanto _lr_ instrui qual a taxa de aprendizado será utilizada.

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

## Treinando o modelo

Agora nosso modelo está pronto para ser treinado. O procedimento será executado por $20$ épocas.

In [17]:
for epoch in range(20):
    model.train()
    
    # Zera os gradientes a cada época (usado no backpropagation).
    #   esse passo é necessário pois a cada vez que o erro é propagado, 
    #   ele é acumulado em vez de ser substituido.
    optimizer.zero_grad()
    
    # Forward pass
    y_pred = model(x_data)
    
    # Computa o erro
    loss = criterion(y_pred, y_data)
    
    # Propaga o erro para as camadas anteriores 
    #    (no caso só temos um, mas seguimos o padrão de redes maiores)
    loss.backward()
    # Atualiza os pesos
    optimizer.step()

## Fazendo predições

Agora que o modelo está treinado, podemos usá-lo para fazer predições dado novos valores de entrada:

In [9]:
new_x = Variable(torch.Tensor([[4.0]]))
y_pred = model(new_x)
print("Valor estimado: ", float(y_pred.data[0][0]))

## Regressão Logística

<img src="assets/regLog.png" width=400px>

In [19]:
# importando pacotes

# https://towardsdatascience.com/logistic-regression-on-mnist-with-pytorch-b048327f8d19
    
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets

## Carregando o dataset

Usamos torchvision.datasets para carregar o dataset Fashion-MNIST. Transforms são ferramentas de normalização, aumento de dados, entre outros. Nesse caso utilizaremos apenas para transformar em vetor.

In [10]:
train_dataset = dsets.FashionMNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.FashionMNIST(root='./data', train=False, transform=transforms.ToTensor())

## Criando um data loader

Conjuntos de dados grandes não podem ser carregados diretamente na memória, principalmente da GPU, por falta de espaço. Para isso, utilizamos data loaders, para carregar _batches_, ou seja, porções de amostras, a cada chamada.

In [21]:
batch_size = 100

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## Criando o modelo

In [22]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

## Definindo os hyperparâmetros e instanciando o modelo

In [23]:
n_iters = 3000
epochs = n_iters / (len(train_dataset) / batch_size)
input_dim = 784
output_dim = 10
lr_rate = 0.001

model = LogisticRegression(input_dim, output_dim)

## Instanciando a classe de Loss

Em seguida definimos nossa função de loss, no caso a entropia cruzada (Cross-Entropy (CE)). Note que CE é praticamente a função Maximum Likelihood Estimation (MLE) que aprendemos na aula passada, com sinal inverso, ou seja, minimizar o BCE é praticamente a mesma coisa que maximizar o MLE.

In [27]:
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then the cross entropy

## Definindo o otimizador

In [25]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

## Treinando o modelo

In [11]:
iter = 0
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iter+=1
        if iter%500==0:
            # calculate Accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1, 28*28))
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct+= (predicted == labels).sum()
            accuracy = 100 * correct.float()/total
            print("Iteration: {}. Loss: {}. Accuracy: {}.".format(iter, loss.item(), accuracy))

## Exercícios

1. Plotar algumas imagens do dataset fashion MNIST para ter uma idea de como são as amostras.
2. Estimar os rótulos das amostras de teste, computar a acurácia e gerar uma matriz de confusão para ver com quais classes cada classe está se confundindo
3. Rodar esse algoritmo utilizando o dataset Kuzushiji-MNIST (KMNIST). Pesquisar para fazer aumento de dados (transform) com horizontal flipping. Plotar algumas imagens desse dataset também. 